In [57]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.optimizers import Adam, SGD
from keras.layers import Dense

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv("../data/HR_comma_sep.csv")

In [5]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [6]:
df.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000
mean,0.612834,0.716102,3.803054,201.050337,3.498233,0.144610,0.238083,0.021268
std,0.248631,0.171169,1.232592,49.943099,1.460136,0.351719,0.425924,0.144281
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [26]:
df['amh200'] = df['average_montly_hours'] / 200
df['tsc10'] = df['time_spend_company'] / 10

In [28]:
sepdf = df[['satisfaction_level', 'last_evaluation', 'number_project', 'amh200', 'tsc10', 'Work_accident', 'promotion_last_5years']]

In [32]:
data = pd.concat([sepdf, pd.get_dummies(df['sales'], prefix="job"), pd.get_dummies(df['salary'], prefix='salary')], axis = 1)

In [38]:
X = data.values
y = df['left']

In [41]:
df['left'].value_counts()
X.shape

(14999, 20)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [85]:
def buildModel(X, y):
    model = Sequential()
    model.add(Dense(1, input_shape=(20,), activation='sigmoid'))
    model.compile(Adam(lr = 0.5), "binary_crossentropy", metrics=["accuracy"])
    model.fit(X, y, epochs=40, verbose=False)
    return model
model = buildModel(X_train, y_train)

In [83]:
y_pred_train = model.predict_classes(X_train)
y_pred_test = model.predict_classes(X_test)

accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

recall_train = accuracy_score(y_train, y_pred_train)
recall_test = accuracy_score(y_test, y_pred_test)

precision_train = precision_score(y_train, y_pred_train)
precision_test = precision_score(y_test, y_pred_test)

f1_train = f1_score(y_train, y_pred_train)
f1_test = f1_score(y_test, y_pred_test)

print("\n")
print("Accuracy: ", accuracy_train, ' --- ', accuracy_test)
print("Recall: ", recall_train, ' --- ', recall_test)
print("Precision: ", precision_train, ' --- ', precision_test)
print("F1: ", f1_train, ' --- ', f1_test)

  32/3000 [..............................] - ETA: 0s0s 

Accuracy:  0.715976331361  ---  0.728
Recall:  0.715976331361  ---  0.728
Precision:  0.440138690598  ---  0.473684210526
F1:  0.558777835318  ---  0.592814371257


In [84]:
from keras.wrappers.scikit_learn import KerasClassifier

In [86]:
def buildModel2():
    model = Sequential()
    model.add(Dense(1, input_shape=(20,), activation='sigmoid'))
    model.compile(Adam(lr = 0.5), "binary_crossentropy", metrics=["accuracy"])
    return model

In [87]:
model = KerasClassifier(build_fn=buildModel2, epochs=10, verbose=0)


In [88]:
from sklearn.model_selection import  KFold, cross_val_score

In [89]:
cv = KFold(5, shuffle=True)
scores = cross_val_score(model, X, y, cv = cv)

In [91]:
scores.mean()

0.75384986103498974

In [92]:
scores.std()

0.03652035169191848

 Conclusion:
 
 The model does not perform better than the benchmark(predicting all 0), so we must look for a better way to solve the problem.